In [41]:
import numpy as np
import pandas as pd

In [42]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [50]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [35]:
list_mz_index = [] ;
# 抹账数据处理
df_data_抹账=df_data_交易[df_data_交易['账户代号'].isin(df_data_交易[df_data_交易['交易金额']<0]['账户代号'].unique())].sort_values(by=['账户代号','交易日期','交易时间'],ascending=True).reset_index() ;
#待剔除数据
tc_idx=set()
for idx in df_data_抹账[df_data_抹账['交易金额']<0].index:
    for d in range(1,5):
        #print(idx)
        if -df_data_抹账.loc[idx,'交易金额']==df_data_抹账.loc[idx-d,'交易金额'] and df_data_抹账.loc[idx,'账户代号']==df_data_抹账.loc[idx-d,'账户代号']\
            and df_data_抹账.loc[idx,'对方账号']==df_data_抹账.loc[idx-d,'对方账号']:
                list_mz_index.append(idx-d) ;
                tc_idx.add(idx)
                tc_idx.add(idx-d)
                break
tc_idx=list(tc_idx)
df_data_交易_去除抹账后=df_data_抹账[~df_data_抹账.index.isin(tc_idx)]
df_data_交易_去除抹账后=pd.concat([df_data_交易_去除抹账后,df_data_交易[df_data_交易['交易金额']>=0]])

In [51]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)

In [58]:
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                break ;

In [68]:
df_data_抹账交易 = df_data_交易.loc[list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index]

In [69]:
df_data_抹账交易.sort_index()

,交易流水序号,账户代号,对方账号,借贷标志,交易金额,交易余额,对方行号,交易日期,交易时间,交易渠道,摘要代号,对方名称长度
22,F1788ACA5169FF97990B8706534EB562,8EB373F073727157,02A4F50406EEA481,0,10000.00,2474.48,0ABE7B2E,2020-03-01,00:23:52,091D584F,A97DA629,9
23,CEB64FED2A87645FCB05A81125C94DA3,8EB373F073727157,02A4F50406EEA481,0,-10000.00,12474.48,0ABE7B2E,2020-03-01,00:23:53,621461AF,149E9677,9
361,5D70376630B872F765E2848C72FC0FEF,2A61369653920C3E,5445E649AE422A38,0,11000.00,12591.49,2BE0C959,2020-03-01,03:40:36,091D584F,A97DA629,6
362,6A44F778641AE5033EA347692C0EB85C,2A61369653920C3E,5445E649AE422A38,0,-11000.00,23591.49,2BE0C959,2020-03-01,03:40:37,621461AF,149E9677,6
818,D6BA9768A39E20A262D071F3E56CDFF2,8EB373F073727157,6B2E0D21D9D3E2D9,0,10000.00,4225.33,2BE0C959,2020-03-01,07:03:36,091D584F,A97DA629,9
819,07FD5568EF51F229BE11C8137CE65296,8EB373F073727157,6B2E0D21D9D3E2D9,0,-10000.00,14225.33,2BE0C959,2020-03-01,07:03:37,621461AF,149E9677,9
825,A4F85399014D24ED18203D36AC49BCC1,8EB373F073727157,6B2E0D21D9D3E2D9,0,10000.00,11725.02,2BE0C959,2020-03-01,07:04:58,091D584F,A97DA629,9
826,053BEEFA92D9AC48E7E86A66E11A2478,8EB373F073727157,6B2E0D21D9D3E2D9,0,-10000.00,21725.02,2BE0C959,2020-03-01,07:04:59,621461AF,149E9677,9
1228,2DE90FDA69FE5DC22D734265E3F6A33F,101F344833993CFB,FB403389BA962862,0,8000.00,42954.03,C89C9E84,2020-03-01,08:03:13,091D584F,A97DA629,9
1229,6598C135627BCB5F902AB44D698A70CF,101F344833993CFB,FB403389BA962862,0,-8000.00,50954.03,C89C9E84,2020-03-01,08:03:14,621461AF,149E9677,9


In [ ]:
df_data_交易_去除抹账后.sort_values(by=["交易日期","交易时间"]).head()

In [ ]:
clicode = "2B4A5A1BE4E40C04"
df1 = df_data_交易_去除抹账后[df_data_交易_去除抹账后["账户代号"] == clicode].sort_values(by=["交易日期","交易时间"])

In [ ]:
df1

In [ ]:
df1[df1["借贷标志"] == 0].groupby("对方账号")["交易金额" , "交易余额"].agg(["mean" , "max" , "min", "count"]).sort_values(by=[("交易金额" , "count")] , ascending=False)
# df1
